In [13]:
import pandas as pd
import numpy as np
from keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tqdm.keras import TqdmCallback
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error


In [2]:
def build_model_regression(lr_,  input_data):
    model = models.Sequential()
    model.add(layers.Dense(32, activation='relu', input_shape=(input_data,)))
    model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = optimizers.RMSprop(learning_rate= lr_),
                loss = 'mse',
                metrics = 'mse')
    return model

In [3]:
tabla0 = pd.read_csv('Temperatures.csv')
time_s = tabla0[['year', 'temperatura']].groupby('year').mean().reset_index()
time_s
years = time_s['year'].values
temps = time_s['temperatura'].values

In [4]:
X_list = []
Y_list = []
year_list = []
for i in range(len(years)-5):
    X_i = temps[i:i+5]
    mean = X_i.mean()
    # X_i = X_i - mean
    std = X_i.std()
    X_i = np.append(X_i, std)
    X_i = np.append(X_i, mean)
    X_list.append(X_i)
    Y_list.append(temps[i+5])
    year_list.append(years[i+5])
X = np.vstack(X_list)
Y = np.vstack(Y_list)

In [5]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [6]:
model = build_model_regression(0.001, 7)
hist = model.fit(x_train, y_train,
          batch_size=200,
          epochs= 50,
          verbose = 0,
          callbacks = TqdmCallback())
y_train_p = model.predict(x_train)
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_train.reshape(-1), mode = 'markers+lines', name='Temps'))
fig.add_trace(go.Scatter(y=y_train_p.reshape(-1), mode = 'markers+lines', name='Predicts'))
fig.show()

100%|██████████| 50/50 [00:01<00:00, 26.18epoch/s, loss=0.0326, mse=0.0326]


3/3 [==============================] - 0s 3ms/step


In [7]:
fig = go.Figure(go.Scatter(y = hist.history['mse']))
fig.show()

In [8]:
y_test_p = model.predict(x_test)
fig = go.Figure(layout_yaxis_range=[15,25])
fig.add_trace(go.Scatter(y=y_test.reshape(-1), mode = 'markers+lines', name='Temps'))
fig.add_trace(go.Scatter(y=y_test_p.reshape(-1), mode = 'markers+lines', name='Predicts'))
fig.show()

1/1 [==============================] - 0s 34ms/step


In [15]:
print(mean_squared_error(y_test.reshape(-1), y_test_p.reshape(-1)))


0.02665022105454226


In [9]:
y_p = model.predict(X)
fig = go.Figure(layout_yaxis_range=[15,25])
fig.add_trace(go.Scatter(x = year_list, y=Y.reshape(-1), mode = 'lines', name='Temps'))
fig.add_trace(go.Scatter(x = year_list, y=y_p.reshape(-1), mode = 'lines', name='Predicts'))
fig.show()

4/4 [==============================] - 0s 2ms/step


In [12]:
dif_min = 10
dif_max = 0
for t,tp in zip(Y.reshape(-1), y_p.reshape(-1)):
    dif = abs(t-tp)
    if dif < dif_min:
        dif_min = dif
    if dif > dif_max:
        dif_max = dif
print('Error máximo:',dif_max)
print('Error mínimo:',dif_min)

Error máximo: 0.45603172302246264
Error mínimo: 0.002366409301757244


In [14]:
print(mean_squared_error(Y.reshape(-1), y_p.reshape(-1)))

0.03131259933028073


In [16]:
model.save('network.h5')